In [1]:
import sys
sys.path.append('../../')

In [2]:
# importnb allows us to load the code from other notebooks
from importnb import Notebook
with Notebook():
    import OfficeAutomation.common as common
    import OfficeAutomation.ac as ac

In [3]:
import crestdsl.model as crest
import crestdsl.simulation as crestsim
import crestdsl.ui as ui

In [8]:
class ACUnitTest(common.TestClass):
        
    def test_setting_influence_need_watt(self):
        acunit = ac.ACUnit()
        sim = crestsim.Simulator(acunit)
        
        for setting, wattage in [("off", 0), ("one", 500), ("two", 700), ("three", 900)]:
            with self.subTest(ac_setting=setting, expected_watt=wattage):
                acunit.setting_in.value = setting
                sim.stabilise()
                self.assertEqual(acunit.need_watt.value, wattage)

    def test_fan_setting_and_electricity_gives_correct_state(self):
        acunit = ac.ACUnit()
        sim = crestsim.Simulator(acunit)
        
        "(setting, electricity, expected_state)"
        params = [
            ("off", 0, acunit.off),
            ("off", 1000, acunit.off),
            ("one", 0, acunit.off),
            ("one", 500, acunit.one), # 35 % efficiency
            ("one", 1000, acunit.one),
            ("two", 0, acunit.off),
            ("two", 500, acunit.off),
            ("two", 700, acunit.two), # 35 % efficiency
            ("two", 1000, acunit.two),
            ("three", 0, acunit.off),
            ("three", 700, acunit.off),
            ("three", 900, acunit.three), # 35 % efficiency
            ("three", 1000, acunit.three),
        ]
        
        for setting, wattage, state in params:
            with self.subTest(ac_setting=setting, input_electricity=wattage, expected_state=state._name):
                acunit.setting_in.value = setting
                acunit.electricity_in.value = wattage
                sim.stabilise()
                sim.advance(1)
                self.assertEqual(acunit.current, state, f"Current state '{acunit.current._name}', expected state '{state._name}'")
        
                
    def test_fan_setting_and_electricity_gives_correct_output(self):
        acunit = ac.ACUnit()
        sim = crestsim.Simulator(acunit)
        
        "(setting, electricity, expected_output)"
        params = [
            ("off", 0, 0),
            ("off", 1000, 0),
            ("one", 0, 0),
            ("one", 500, 500*0.35), # 35 % efficiency
            ("one", 1000, 500*0.35),
            ("two", 0, 0),
            ("two", 500, 0),
            ("two", 700, 700*0.35), # 35 % efficiency
            ("two", 1000, 700*0.35),
            ("three", 0, 0),
            ("three", 700, 0),
            ("three", 900, 900*0.35), # 35 % efficiency
            ("three", 1000, 900*0.35),
        ]
        
        for setting, wattage, expected_output in params:
            with self.subTest(ac_setting=setting, input_electricity=wattage, expected_output=expected_output):
                acunit.setting_in.value = setting
                acunit.electricity_in.value = wattage
                sim.stabilise()
                sim.advance(1)
                self.assertEqual(acunit.output.value, expected_output)
        
        
# ACUnitTest.runall()    
# crest.Validator(ACUnit()).check_all()

# ui.plot(ACUnit())

...
----------------------------------------------------------------------
Ran 3 tests in 0.745s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>